In [2]:
from general import *
from generation_stm import *
from storage_stm import *
from system_stm import *
from loaddata import calculate_greenhouse_load
from one_D_model import *
from datetime import datetime, timedelta
import os, shutil
import matplotlib.dates as mdates

In [3]:
geometry_data = {
    '3-1':   {'tank_cost':  None,  'insulation_cost':  None,  'exergy_efficiency':  None},
    '2.5-1': {'tank_cost':  None,  'insulation_cost':  None,  'exergy_efficiency':  None},
    '2-1':   {'tank_cost':  None,  'insulation_cost':  None,  'exergy_efficiency':  None},
    '1.5-1': {'tank_cost':  None,  'insulation_cost':  None,  'exergy_efficiency':  None},
    '1-1':   {'tank_cost':  None,  'insulation_cost':  None,  'exergy_efficiency':  None},
    '1-1.5': {'tank_cost':  None,  'insulation_cost':  None,  'exergy_efficiency':  None},
    '1-2':   {'tank_cost':  None,  'insulation_cost':  None,  'exergy_efficiency':  None},
    '1-2.5': {'tank_cost':  None,  'insulation_cost':  None,  'exergy_efficiency':  None},
    '1-3':   {'tank_cost':  None,  'insulation_cost':  None,  'exergy_efficiency':  None},
}

In [4]:
# get load data
current_dir = os.getcwd()
original_weather_file = "C:\\Users\\tljohn\\Desktop\\RPB Micmaq\\code\\presque_isle_main_tmy.csv"

num_copies = 1
Kfiles = []
# creating multiple copies of the original weather file
for i in range(num_copies):
    new_file = os.path.join(current_dir, f'presque_isle_main_tmy_{i}.csv')
    shutil.copyfile(original_weather_file, new_file)
    Kfiles.append(new_file)
weather_files = Kfiles
greenhouse_load = calculate_greenhouse_load(weather_files)
ambient_temps = greenhouse_load['T_amb']
T_greenhouse = greenhouse_load['T_greenhouse']
greenhouse_load = greenhouse_load[['NetLoad_MW']]
ambient_temps = ambient_temps[1]
greenhouse_load_arr = greenhouse_load.to_numpy()
greenhouse_load_arr = greenhouse_load_arr.flatten()
date_range = pd.date_range(start='2033-01-01 00:30:00', periods=len(greenhouse_load_arr), freq='h')
greenhouse_load = pd.DataFrame({'load_MW': greenhouse_load_arr}, index=date_range)
greenhouse_load.index.name = 'Date/time'

ambient_temps = pd.DataFrame(ambient_temps, index=greenhouse_load.index, columns=['T_amb'])
ambient_temps.index.name = 'Date/time'

Total Q_net for the year: -287.74 MWh


c:\Users\tljohn\Desktop\RPB Micmaq\code\loaddata.py:123: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start='2023-01-01', periods=num_hours, freq=f'{timestep_hours}H')
C:\Users\tljohn\AppData\Local\Temp\ipykernel_21072\2119950351.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ambient_temps = ambient_temps[1]


In [ ]:
max_greenhouse_load = np.max(greenhouse_load_arr)
power_block_size = max_greenhouse_load

np.float64(0.12569766832453413)

In [ ]:
# TEA system setup
ratio_MW2acres = {'SAT': 0.24, 'fixed': 0.35}
pv_type = 'fixed'
pv_dc2ac = 1.2 
POI_limit = None  
analysis_period = 25  
ITC = 0.3  

# Setup Site and Systems
NSTTF_site = Site('NSTTF', Weather_Data(Kfiles), -7, POI_limit=POI_limit)
power_block = power_block_size # MW
PV_capacity_AC = 0.1 
PV_capacity_DC = PV_capacity_AC * pv_dc2ac  
ASGARD_PV = PV_System(
    name='ASGARD_PV',
    site=NSTTF_site,
    capacity_MW_DC=PV_capacity_DC,
    PV_array_type=pv_type,
    PV_tilt=30,
    PV_azimuth=180,
    ratio_DC2AC=pv_dc2ac,
    off_grid_operation=False,
    power_priority_load_MW_AC=None
)

TES_capacity_e = 1.1 # MWh
resistive_charge_rate = PV_capacity_DC

aspect_ratios = list(geometry_data.keys())

all_timeseries = {}

for ar in aspect_ratios:
    geo = geometry_data[ar]
    ex_eff = geo['exergy_efficiency']
    insulation_cost = geo['insulation_cost']
    tank_cost = geo['tank_cost']

    # e.g. percent_heat_loss_daily = (1 - ex_eff) 
    heat_loss_daily = (1.0 - ex_eff) # /24 ? *************

    # redefine TES system for this aspect ratio
    ASGARD_TES = TES_System_STM(
        name=f'ASGARD_TES_{ar}',
        site=NSTTF_site,
        TES_type=ar,
        insulation_cost = insulation_cost,
        tank_cost = tank_cost,
        capacity_MWh_t=TES_capacity_e / 0.98,
        power_rating_MW_e=power_block,
        power_minimum_MW_e=0,
        efficiency_rating_t2e=0.98,
        percent_discharge_depth=100,
        percent_heat_loss_daily=heat_loss_daily,
        charge_rate_CSP_MW_t=0,
        charge_efficiency_t2TES=1,
        charge_rate_resistive_MW_e=resistive_charge_rate,
        charge_efficiency_e2TES=0.98,
        systems_charging=['ASGARD_PV'],
        start_full=False,
        off_grid_operation=True,
        DOE_2030_targets=False
    )
 
    systems = [ASGARD_PV, ASGARD_TES]
    NSTTF = System(
        load_MW=greenhouse_load,
        systems_load_order=systems,
        analysis_period=analysis_period,
        ITC=ITC
    )

    # run timeseries and store
    ts = NSTTF.timeseries()
    all_timeseries[ar] = ts


In [ ]:

# TEA system setup
ratio_MW2acres = {'SAT': 0.24, 'fixed': 0.35}
pv_type = 'fixed'
pv_dc2ac = 1.2 
POI_limit = None  
analysis_period = 25  
ITC = 0.3  

# Setup Site and Systems
NSTTF_site = Site('NSTTF', Weather_Data(Kfiles), -7, POI_limit=POI_limit)
power_block = 0.2 # MW
PV_capacity_AC = 0.1 
PV_capacity_DC = PV_capacity_AC * pv_dc2ac  
ASGARD_PV = PV_System(
    name='ASGARD_PV',
    site=NSTTF_site,
    capacity_MW_DC=PV_capacity_DC,
    PV_array_type=pv_type,
    PV_tilt=30,
    PV_azimuth=180,
    ratio_DC2AC=pv_dc2ac,
    off_grid_operation=False,
    power_priority_load_MW_AC=None
)

TES_capacity_e = 1.1 # MWh
resistive_charge_rate = PV_capacity_DC

# for parametric, make sure TES type/efficiency are set as pair (7 runs)

ASGARD_TES = TES_System_STM(
    name='ASGARD_TES',
    site=NSTTF_site,
    # TES type
    TES_type, # set up to loop through each TES type (aspect ratio)
    capacity_MWh_t=TES_capacity_e / 0.98,
    power_rating_MW_e=power_block, # ensure this is larger than load peak (see gh)
    power_minimum_MW_e=0,
    efficiency_rating_t2e=.98, 
    percent_discharge_depth=100,
    percent_heat_loss_daily=0, # (1- exergy efficiency) based on TES type # ensure divided by 24
    charge_rate_CSP_MW_t=0,
    charge_efficiency_t2TES=1, 
    charge_rate_resistive_MW_e=resistive_charge_rate,
    charge_efficiency_e2TES=0.98,
    systems_charging=['ASGARD_PV'],
    start_full=False,
    off_grid_operation=True,
    DOE_2030_targets=False
)

systems = [ASGARD_PV, ASGARD_TES]
NSTTF = System(
    load_MW=greenhouse_load,
    systems_load_order=systems,
    analysis_period=analysis_period,
    ITC=ITC
)
tea_timeseries = NSTTF.timeseries